In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

/home/uichan/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import numpy as numpy
import matplotlib.pyplot as plt
from tqdm import tqdm

In [3]:
import string
import unidecode
import random

In [4]:
all_char = string.printable
all_char_len = len(all_char)

file = unidecode.unidecode(open("data/tiny-shakespeare.txt").read())
file_len = len(file)
print(file_len)

print(all_char)
print(all_char_len)

1115394
0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ 	

100


In [5]:
def char_tensor(txt):
    len_txt = len(txt)
    tensor = torch.zeros(len_txt)

    for c in range(len_txt):
        tensor[c] = all_char.index(txt[c])
        
    return tensor

print(char_tensor("abc"))

tensor([10., 11., 12.])


In [12]:
def random_chunk(chunk_len = 1000):
    start_index = random.randint(0,file_len - chunk_len)
    end_index  = start_index + chunk_len
    
    return file[start_index:end_index]

def make_set(To_tensor = True):
    chunk = random_chunk()
    if To_tensor:
        chunk = char_tensor(chunk)
    inp = chunk[:-1]
    target = chunk[1:]

    return inp, target

In [7]:
class RNN(nn.Module):
    def __init__(self, input_size, output_size, hidden_size, embedding_size, num_layer = 1):
        super().__init__()

        #size info
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_size = hidden_size
        self.num_layer = num_layer
        self.embedding_size = embedding_size

        #model
        self.encoder = nn.Embedding(input_size, embedding_size)
        self.rnn     = nn.RNN(embedding_size, hidden_size, num_layer) #여기 함수 이름을 바꾸면 됨 RNN <-> GRU
        self.decoder = nn.Linear(hidden_size, output_size)
    
    def forward(self, input, hidden):
        out = self.encoder(input)
        out,hidden = self.rnn(out,hidden)
        out = self.decoder(out)
        return out, hidden

    def init_hidden(self):
        batch_size = 64
        hidden = torch.zeros(self.num_layer, self.hidden_size, batch_size)
        return hidden


In [10]:
epoch = 1000
lr = 0.0002
chunk_len = 1000
model = RNN(all_char_len,all_char_len,20,50,2)
loss_func = nn.MSELoss()
optimizer = optim.Adam(model.parameters(),lr = lr)

inp = char_tensor("A")
hidden = model.init_hidden()
out,hidden = (inp,hidden)

In [13]:
loss_array = []

for i in tqdm(range(epoch)):
    model.zero_grad()
    total_loss = 0
    inp, label = make_set()
    hidden = model.init_hidden()

    for j in range(chunk_len - 1):
        input_ = inp[j]
        target = label[j]

        output,hidden = model.forward(input_,hidden)
        loss = loss_func(output,target)
        total_loss += loss
        
    total_loss.backward()
    optimizer.step()
    loss_array.append(total_loss.cpu().detach().numpy())

  0%|          | 0/1000 [00:00<?, ?it/s]


RuntimeError: Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.FloatTensor instead (while checking arguments for embedding)